In [ ]:
import pandas as pd
import numpy as np
import os
from dshortcuts.utils import open_pickle, save_pickle
# from operator import itemgetter
import geocoder

In [ ]:
os.listdir('../input')

In [ ]:
# open prepared dtypes to save memory
trainDtypes = open_pickle(os.path.join('../input', 'trainDtypes.pickle'))
itemCategoriesDtypes = open_pickle(os.path.join('../input', 'itemCategoriesDtypes.pickle'))

In [ ]:
train = pd.read_csv('../input/sales_train.csv', dtype=trainDtypes, parse_dates=['date'])
itemCategories = pd.read_csv('../input/item_categories.csv', dtype=itemCategoriesDtypes)
test = pd.read_csv('../input/test.csv')
shops = pd.read_csv('../input/shops.csv')
items = pd.read_csv('../input/items.csv')

In [ ]:
train.set_ind

##### Categories
1. Are they broad enough?
2. Are they descriptive enough?

In [ ]:
itemCategories.head()

too descriptive. At first look
we may try to decouple broad categories by `split('-')`

In [ ]:
test.head()

In [ ]:
train = train.merge(items.drop('item_name', axis=1), on='item_id', how='left')

#### Obvious features: 

- ~~day~~
- month
- quarter
- year
- ~~dayofweek~~
- ~~is workingday~~
- ~~delta from every 22th day of month (~salary day in Rus)~~
- ~~delta from every 7th day of month (~pre-salary day in Rus)~~

I considered that in test set there is only a month that we know from a datetime features. It means that we can get only month+ features

In [ ]:
# train['day'] = train['date'].dt.day
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train['quarter'] = train['date'].dt.quarter
# train['dayofweek'] = train['date'].dt.weekday
# train['is_weekend'] = (train['dayofweek'] >= 5).astype('int8')

In [ ]:
train.head()

#### Sales & Price features: (think more...)

**H1**:
! People buy things when the price goes down

- 1quart price in previous month (shop-item)
- 3quart price in previous month (shop-item)
- price delta in prev month (shop-item)
- price delta over 2 months (shop-item)
- monthly sales by {shop_id, category_id}
- monthly sales by {shop_id, item_id}
- monthly sales by {category_id}

Probably it worth to make a **H2**:
! People buy when a shop signals about sales (no matter which item)

In [ ]:
shop_item = train.groupby(['date_block_num', 'shop_id', 'item_id'])

In [ ]:
q25 = shop_item.item_price.quantile(.25)
q75 = shop_item.item_price.quantile(.75)

In [ ]:
q25.reset_index()

In [ ]:
q25.reset_index().date_block_num + 1

In [ ]:
train.merge(
    q25.reset_index().rename({'item_price': 'mon_price_1qrt'}, axis=1), 
    on=['date_block_num', 'shop_id', 'item_id'])

In [ ]:
q25.reset_index().rename({'item_price': 'mon_price_1qrt'}, axis=1)

In [ ]:
pd.DataFrame.rename?

In [ ]:
shop_item.item_price.quantile(.75)

In [ ]:
((shop_item.item_price.quantile(.25) - shop_item.item_price.quantile(.75)) != 0)

#### Geographical features
1. Get the city from the shop info
2. Try to get the lat lon of the shop with map api ?

-------------

- what's next?

In [ ]:
def extract_city(value):
    return value.split()[0].strip('!')

def yandex_latlong(city):
    return geocoder.google(city).latlng


shops['city'] = shops['shop_name'].apply(extract_city)

In [ ]:
# because of the error with geocoder.yandex, i took ready latlong from here:
# https://www.kaggle.com/bourov/yada-yet-another-e-da-by-geo-location-of-shops/data?select=coordinates.csv
coordinates = pd.read_csv('../input/coordinates.csv', index_col=0)
coordinates.head()

In [ ]:
shops = shops.merge(coordinates, left_on='shop_id', right_index=True)
shops

In [ ]:
shops.groupby('city').size().describe()

##### C&C: Cleaning &Clustering
Let's try to cleanup the NaNs, indicate them with a new column, and add clusters with kNN (aka Regions)

In [ ]:
train.groupby(['date_block_num', 'shop_id', 'item_id']).item_cnt_day.sum()

#### Validation

- is it just a timewise split?
- do we have new shops / items in test set? (if yes - we have to mimic this in validation strategy also)

In [ ]:
# shops in test but not in train (unseen)
_mask = ~ np.isin(test.shop_id.unique(), train.shop_id.unique())
test.shop_id.unique()[_mask]

In [ ]:
# items in test but not in train (unseen)
_mask = ~ np.isin(test.item_id.unique(), train.item_id.unique())
test.item_id.unique()[_mask].shape,  test.item_id.unique().shape

Let's do the same with a validation set

In [ ]:
_mask = ~ np.isin(valid.shop_id.unique(), train[train.date_block_num < 33].shop_id.unique())
valid.shop_id.unique()[_mask]

In [ ]:
_mask = ~ np.isin(valid.item_id.unique(), train[train.date_block_num < 33].item_id.unique())
valid.item_id.unique()[_mask].shape, valid.item_id.unique().shape

Well, the test set is roughly the same as validation. We can use validation set as `date_block_num == 33` for sure

Probably it's a good idea to exclude `shop_id == 36` to mimic a test set (the last has no new shops)

One more **TODO**:
check if we can use 2 folds: 33's and 32's blocks

In [ ]:
valid = train[train['date_block_num'] == (train['date_block_num'].max())]

In [ ]:
(test['shop_id'].isin(train['shop_id']) == False).sum(), \
(test['item_id'].isin(train['item_id']) == False).sum()

In [ ]:
test.head()